In [1]:
import edu.umd.cloud9.collection.XMLInputFormat
import org.apache.spark.sql.{ Dataset, SparkSession,Row }
import org.apache.hadoop.io.{ Text, LongWritable }
import org.apache.hadoop.conf.Configuration

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1603053752604)
SparkSession available as 'spark'


import edu.umd.cloud9.collection.XMLInputFormat
import org.apache.spark.sql.{Dataset, SparkSession, Row}
import org.apache.hadoop.io.{Text, LongWritable}
import org.apache.hadoop.conf.Configuration


In [2]:
def loadMedline(spark: SparkSession, path: String) = {
    import spark.implicits._
    @transient val conf = new Configuration()
    conf.set(XMLInputFormat.START_TAG_KEY, "<MedlineCitation ")
    conf.set(XMLInputFormat.END_TAG_KEY, "</MedlineCitation>")
    val sc = spark.sparkContext
    val in = sc.newAPIHadoopFile(path, classOf[XMLInputFormat],
                                classOf[LongWritable], classOf[Text], conf)
    in.map(line => line._2.toString).toDS()
}
val medlineRaw = loadMedline(spark, "Data/medline_data")

loadMedline: (spark: org.apache.spark.sql.SparkSession, path: String)org.apache.spark.sql.Dataset[String]
medlineRaw: org.apache.spark.sql.Dataset[String] = [value: string]


In [3]:
import scala.xml._
val rawXml = medlineRaw.take(1)(0)
val elem = XML.loadString(rawXml)

import scala.xml._
rawXml: String =
<MedlineCitation Owner="PIP" Status="MEDLINE">
<PMID Version="1">12255379</PMID>
<DateCreated>
<Year>1980</Year>
<Month>01</Month>
<Day>03</Day>
</DateCreated>
<DateCompleted>
<Year>1980</Year>
<Month>01</Month>
<Day>03</Day>
</DateCompleted>
<DateRevised>
<Year>2013</Year>
<Month>02</Month>
<Day>19</Day>
</DateRevised>
<Article PubModel="Print">
<Journal>
<ISSN IssnType="Print">0002-9955</ISSN>
<JournalIssue CitedMedium="Print">
<Volume>159</Volume>
<Issue>3</Issue>
<PubDate>
<Year>1955</Year>
<Month>Sep</Month>
<Day>17</Day>
</PubDate>
</JournalIssue>
<Title>Journal of the American Medical Association</Title>
<ISOAbbreviation>J Am Med Assoc</ISOAbbreviation>
</Journal>
<ArticleTitle>Association of maternal and fetal factors with development of menta...

In [4]:
elem.label

res0: String = MedlineCitation


In [5]:
elem.attributes

res1: scala.xml.MetaData =  Status="MEDLINE" Owner="PIP"


In [6]:
elem \ "MeshHeadingList"

res2: scala.xml.NodeSeq =
NodeSeq(<MeshHeadingList>
<MeshHeading>
<DescriptorName UI="D001519" MajorTopicYN="N">Behavior</DescriptorName>
</MeshHeading>
<MeshHeading>
<DescriptorName UI="D000013" MajorTopicYN="N">Congenital Abnormalities</DescriptorName>
</MeshHeading>
<MeshHeading>
<DescriptorName UI="D006233" MajorTopicYN="N">Disabled Persons</DescriptorName>
</MeshHeading>
<MeshHeading>
<DescriptorName UI="D004194" MajorTopicYN="N">Disease</DescriptorName>
</MeshHeading>
<MeshHeading>
<DescriptorName UI="D008607" MajorTopicYN="Y">Intellectual Disability</DescriptorName>
</MeshHeading>
<MeshHeading>
<DescriptorName UI="D007360" MajorTopicYN="N">Intelligence</DescriptorName>
</MeshHeading>
<MeshHeading>
<DescriptorName UI="D008431" MajorTopicYN="Y">Maternal-Fetal Exchange</DescriptorNa...

In [7]:
elem \\ "MeshHeading"

res3: scala.xml.NodeSeq =
NodeSeq(<MeshHeading>
<DescriptorName UI="D001519" MajorTopicYN="N">Behavior</DescriptorName>
</MeshHeading>, <MeshHeading>
<DescriptorName UI="D000013" MajorTopicYN="N">Congenital Abnormalities</DescriptorName>
</MeshHeading>, <MeshHeading>
<DescriptorName UI="D006233" MajorTopicYN="N">Disabled Persons</DescriptorName>
</MeshHeading>, <MeshHeading>
<DescriptorName UI="D004194" MajorTopicYN="N">Disease</DescriptorName>
</MeshHeading>, <MeshHeading>
<DescriptorName UI="D008607" MajorTopicYN="Y">Intellectual Disability</DescriptorName>
</MeshHeading>, <MeshHeading>
<DescriptorName UI="D007360" MajorTopicYN="N">Intelligence</DescriptorName>
</MeshHeading>, <MeshHeading>
<DescriptorName UI="D008431" MajorTopicYN="Y">Maternal-Fetal Exchange</DescriptorName>
</MeshHe...

In [8]:
(elem \\ "DescriptorName").map(_.text)

res4: scala.collection.immutable.Seq[String] = List(Behavior, Congenital Abnormalities, Disabled Persons, Disease, Intellectual Disability, Intelligence, Maternal-Fetal Exchange, Personality, Pregnancy, Pregnancy Complications, Psychology, Reproduction, Research)


In [9]:
def majorTopics(record:String):Seq[String] = {
    val elem = XML.loadString(record)
    val dn = elem \\"DescriptorName"
    val mt = dn.filter(n => (n \ "@MajorTopicYN").text=="Y")
    mt.map(n => n.text)
}

majorTopics: (record: String)Seq[String]


In [10]:
val medline = medlineRaw.map(majorTopics)
medline.cache()
medline.take(1)(0)

medline: org.apache.spark.sql.Dataset[Seq[String]] = [value: array<string>]
res5: Seq[String] = List(Intellectual Disability, Maternal-Fetal Exchange, Pregnancy Complications)


In [11]:
medline.count()
val topics = medline.flatMap(mesh => mesh).toDF("topic")
topics.createOrReplaceTempView("topics")
val topicDist = spark.sql("""
SELECT topic, COUNT(*) cnt
FROM topics
GROUP BY topic
ORDER BY cnt DESC""")
topicDist.count()

topics: org.apache.spark.sql.DataFrame = [topic: string]
topicDist: org.apache.spark.sql.DataFrame = [topic: string, cnt: bigint]
res6: Long = 7699


In [12]:
topicDist.show()

+--------------------+----+
|               topic| cnt|
+--------------------+----+
|             Disease|1202|
|           Neoplasms| 983|
|        Tuberculosis| 950|
|               Blood| 518|
|          Anesthesia| 379|
| Wounds and Injuries| 370|
|            Medicine| 322|
|Nervous System Ph...| 318|
|Antibiotics, Anti...| 298|
|         Body Fluids| 284|
| Biomedical Research| 283|
|               Brain| 276|
|Congenital Abnorm...| 271|
|Cardiovascular Sy...| 261|
|         Vaccination| 251|
|        Bacteriology| 247|
|               Urine| 245|
|Anti-Bacterial Ag...| 239|
|              Niacin| 236|
|             History| 235|
+--------------------+----+
only showing top 20 rows



In [13]:
val list = List(1,2,3)
val combs = list.combinations(2)
combs.foreach(println)

List(1, 2)
List(1, 3)
List(2, 3)


list: List[Int] = List(1, 2, 3)
combs: Iterator[List[Int]] = empty iterator


In [14]:
val combs = list.reverse.combinations(2)
combs.foreach(println)
List(3,2) == List(2,3)

List(3, 2)
List(3, 1)
List(2, 1)


combs: Iterator[List[Int]] = empty iterator
res9: Boolean = false


In [15]:
val topicPairs = medline.flatMap(t => {
    t.sorted.combinations(2)
}).toDF("pairs")
topicPairs.createOrReplaceTempView("topic_pairs")
val cooccurs = spark.sql("""
SELECT pairs, COUNT(*) cnt
FROM topic_pairs
GROUP BY pairs""")
cooccurs.cache()
cooccurs.count()

topicPairs: org.apache.spark.sql.DataFrame = [pairs: array<string>]
cooccurs: org.apache.spark.sql.DataFrame = [pairs: array<string>, cnt: bigint]
res10: Long = 72560


In [16]:
cooccurs.createOrReplaceTempView("cooccurs")
spark.sql("""SELECT pairs, cnt
FROM cooccurs
ORDER BY cnt DESC
LIMIT 10""").collect().foreach(println)

[WrappedArray(Antibiotics, Antitubercular, Dermatologic Agents),195]
[WrappedArray(Analgesia, Anesthesia),181]
[WrappedArray(Analgesia, Anesthesia and Analgesia),179]
[WrappedArray(Anesthesia, Anesthesia and Analgesia),177]
[WrappedArray(Anesthesia, Anesthesiology),153]
[WrappedArray(Nervous System Physiological Phenomena, Reflex),151]
[WrappedArray(Body Fluids, Urine),149]
[WrappedArray(Antibodies, Antigens),133]
[WrappedArray(Niacin, Tuberculosis),131]
[WrappedArray(Isoniazid, Niacin),129]


In [17]:
import java.nio.charset.StandardCharsets
import java.security.MessageDigest

def hashId(str: String): Long = {
    val bytes = MessageDigest.getInstance("MD5").
    digest(str.getBytes(StandardCharsets.UTF_8))
    (bytes(0) & 0xFFL) |
    ((bytes(1) & 0xFFL) << 8) |
    ((bytes(2) & 0xFFL) << 16) |
    ((bytes(3) & 0xFFL) << 24) |
    ((bytes(4) & 0xFFL) << 32) |
    ((bytes(5) & 0xFFL) << 40) |
    ((bytes(6) & 0xFFL) << 48) |
    ((bytes(7) & 0xFFL) << 56) 
}

import java.nio.charset.StandardCharsets
import java.security.MessageDigest
hashId: (str: String)Long


In [18]:
import org.apache.spark.sql.Row

val vertices = topics.map{ case Row(topic: String) =>
    (hashId(topic),topic)}.toDF("hash", "topic")

import org.apache.spark.sql.Row
vertices: org.apache.spark.sql.DataFrame = [hash: bigint, topic: string]


In [19]:
import org.apache.spark.graphx._
val edges = cooccurs.map{case Row(topics: Seq[_], cnt: Long) =>
val ids = topics.map(_.toString).map(hashId).sorted
Edge(ids(0),ids(1),cnt)
}

import org.apache.spark.graphx._
edges: org.apache.spark.sql.Dataset[org.apache.spark.graphx.Edge[Long]] = [srcId: bigint, dstId: bigint ... 1 more field]


In [20]:
val vertexRDD = vertices.rdd.map{
    case Row(hash: Long, topic: String) => (hash, topic)
}
val topicGraph = Graph(vertexRDD, edges.rdd)
topicGraph.cache()

vertexRDD: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[84] at map at <console>:41
topicGraph: org.apache.spark.graphx.Graph[String,Long] = org.apache.spark.graphx.impl.GraphImpl@6a9fb0c0
res12: org.apache.spark.graphx.Graph[String,Long] = org.apache.spark.graphx.impl.GraphImpl@6a9fb0c0


In [21]:
vertexRDD.count()

res13: Long = 82195


In [22]:
topicGraph.vertices.count()

res14: Long = 7699


In [23]:
val connectedComponentGraph = topicGraph.connectedComponents()

connectedComponentGraph: org.apache.spark.graphx.Graph[org.apache.spark.graphx.VertexId,Long] = org.apache.spark.graphx.impl.GraphImpl@7757f20d


In [24]:
val componentDF = connectedComponentGraph.vertices.toDF("vid","cid")
val componentCounts = componentDF.groupBy("cid").count()
componentCounts.orderBy($"count".desc).show()

+--------------------+-----+
|                 cid|count|
+--------------------+-----+
|-9215470674759766104| 7572|
| 1765411469112156596|    3|
| 3608770526546285755|    3|
| 6617131068913763726|    2|
| 2277126264016938007|    2|
|-5293832349673873297|    2|
|-7323847459013211825|    2|
|-4492732731552733030|    2|
|-7059326802402716265|    2|
|  731949936574312042|    2|
|-2551964588563249305|    2|
| 5826763844776297056|    2|
| 7056830101856982450|    1|
| 3397189176630022717|    1|
| 6103591236614385837|    1|
| 2893116236831620270|    1|
| 3697065331967883755|    1|
| 3757970659535796638|    1|
|  -52388043055878779|    1|
| 1993830135517246392|    1|
+--------------------+-----+
only showing top 20 rows



componentDF: org.apache.spark.sql.DataFrame = [vid: bigint, cid: bigint]
componentCounts: org.apache.spark.sql.DataFrame = [cid: bigint, count: bigint]


In [25]:
val topicComponentDF = topicGraph.vertices.innerJoin(
    connectedComponentGraph.vertices) {
    (topicid, name, componentId) => (name, componentId.toLong)
    }.values.toDF("topic","cid")

topicComponentDF: org.apache.spark.sql.DataFrame = [topic: string, cid: bigint]


In [26]:
topicComponentDF.where("cid = 1765411469112156596").show()

+-----------+-------------------+
|      topic|                cid|
+-----------+-------------------+
|Herpestidae|1765411469112156596|
|   Isospora|1765411469112156596|
|   Coccidia|1765411469112156596|
+-----------+-------------------+



In [27]:
topicDist.createOrReplaceTempView("topic_dist")
spark.sql("""
SELECT cnt, COUNT(*) dist
FROM topic_dist
GROUP BY cnt
ORDER BY dist DESC
LIMIT 10""").show()

+---+----+
|cnt|dist|
+---+----+
|  1|2347|
|  2|1136|
|  3| 724|
|  4| 487|
|  5| 379|
|  6| 307|
|  7| 199|
|  8| 193|
|  9| 169|
| 10| 141|
+---+----+



In [28]:
val campy = spark.sql("""
SELECT *
FROM topic_dist
WHERE topic LIKE '%Herpestidae%'""")
campy.show()

+-----------+---+
|      topic|cnt|
+-----------+---+
|Herpestidae|  1|
+-----------+---+



campy: org.apache.spark.sql.DataFrame = [topic: string, cnt: bigint]


In [29]:
val degrees: VertexRDD[Int] = topicGraph.degrees.cache()
degrees.map(_._2).stats()

degrees: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[262] at RDD at VertexRDD.scala:57
res19: org.apache.spark.util.StatCounter = (count: 7596, mean: 19.104792, stdev: 40.490526, max: 1482.000000, min: 1.000000)


In [30]:
val namesAndDegrees = degrees.innerJoin(topicGraph.vertices) {
    (topicId, degree, name) => (name, degree.toInt)
}.values.toDF("topic","degree")

namesAndDegrees: org.apache.spark.sql.DataFrame = [topic: string, degree: int]


In [31]:
namesAndDegrees.orderBy($"degree".desc).show()

+--------------------+------+
|               topic|degree|
+--------------------+------+
|             Disease|  1482|
|           Neoplasms|   918|
|               Blood|   706|
|        Tuberculosis|   665|
| Wounds and Injuries|   449|
|         Body Fluids|   396|
|               Brain|   391|
|Congenital Abnorm...|   371|
|Antibiotics, Anti...|   370|
|          Anesthesia|   356|
|            Research|   354|
|               Urine|   345|
|            Syndrome|   344|
|        Bacteriology|   339|
|Diagnosis, Differ...|   333|
|Physiological Pro...|   328|
| Biomedical Research|   324|
|Cardiovascular Sy...|   323|
|              Plants|   309|
|       Skin Diseases|   309|
+--------------------+------+
only showing top 20 rows



In [32]:
val T = medline.count()

T: Long = 30000


In [33]:
val topicDistRdd = topicDist.map{
    case Row(topic: String, cnt: Long) => (hashId(topic),cnt)
}.rdd

topicDistRdd: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[299] at rdd at <console>:43


In [34]:
val topicDistGraph = Graph(topicDistRdd, topicGraph.edges)

topicDistGraph: org.apache.spark.graphx.Graph[Long,Long] = org.apache.spark.graphx.impl.GraphImpl@b76287f


In [35]:
def chiSq(YY: Long, YB: Long, YA: Long, T: Long): Double = {
    val NB = T-YB
    val NA = T-YA
    val YN = YA-YY
    val NY = YB-YY
    val NN = T-NY-YN-YY
    val inner = math.abs(YY * NN - YN * NY) - T / 2.0
    T * math.pow(inner,2)/(YA*NA*YB*NB)
}

chiSq: (YY: Long, YB: Long, YA: Long, T: Long)Double


In [36]:
val chiSquaredGraph = topicDistGraph.mapTriplets(triplet => {
    chiSq(triplet.attr, triplet.srcAttr, triplet.dstAttr, T)})
    chiSquaredGraph.edges.map(x => x.attr).stats()

chiSquaredGraph: org.apache.spark.graphx.Graph[Long,Double] = org.apache.spark.graphx.impl.GraphImpl@2b21f1f2
res21: org.apache.spark.util.StatCounter = (count: 72560, mean: 315.119299, stdev: 1354.301605, max: 29096.810219, min: 0.000000)


In [37]:
val interesting = chiSquaredGraph.subgraph(triplet => triplet.attr > 19.5)
interesting.edges.count

interesting: org.apache.spark.graphx.Graph[Long,Double] = org.apache.spark.graphx.impl.GraphImpl@20a74ac7
res22: Long = 36938


In [38]:
val interestingComponentGraph = interesting.connectedComponents()
val icDF = interestingComponentGraph.vertices.toDF("vid","cid")
val icCountDF = icDF.groupBy("cid").count()
icCountDF.count()

interestingComponentGraph: org.apache.spark.graphx.Graph[org.apache.spark.graphx.VertexId,Double] = org.apache.spark.graphx.impl.GraphImpl@3bd75833
icDF: org.apache.spark.sql.DataFrame = [vid: bigint, cid: bigint]
icCountDF: org.apache.spark.sql.DataFrame = [cid: bigint, count: bigint]
res23: Long = 125


In [39]:
icCountDF.orderBy($"count".desc).show()

+--------------------+-----+
|                 cid|count|
+--------------------+-----+
|-9215470674759766104| 7561|
| 1765411469112156596|    3|
| 3608770526546285755|    3|
|-2551964588563249305|    2|
|  731949936574312042|    2|
|-7323847459013211825|    2|
|-4492732731552733030|    2|
|-5293832349673873297|    2|
|-7285954876885413962|    2|
| 6617131068913763726|    2|
|-7059326802402716265|    2|
| 2277126264016938007|    2|
| 5826763844776297056|    2|
| 2893116236831620270|    1|
| 7303824060009723512|    1|
| 7056830101856982450|    1|
| 6103591236614385837|    1|
|  -52388043055878779|    1|
| 1993830135517246392|    1|
|-4558125522602794424|    1|
+--------------------+-----+
only showing top 20 rows



In [40]:
val interestingDegrees = interesting.degrees.cache()
interestingDegrees.map(_._2).stats()

interestingDegrees: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[484] at RDD at VertexRDD.scala:57
res25: org.apache.spark.util.StatCounter = (count: 7587, mean: 9.737182, stdev: 10.634602, max: 247.000000, min: 1.000000)


In [41]:
interestingDegrees.innerJoin(topicGraph.vertices){
    (topicId, degree, name) => (name, degree)
}.values.toDF("topic","degree").orderBy($"degree".desc).show()

+--------------------+------+
|               topic|degree|
+--------------------+------+
|             Disease|   247|
|           Neoplasms|   212|
|              Plants|   134|
|     Fractures, Bone|   103|
|        Bacteriology|    94|
|             Viruses|    88|
|            Bacteria|    83|
|Congenital Abnorm...|    82|
|            Arteries|    79|
|          Microscopy|    77|
| Wounds and Injuries|    77|
|Anesthesia and An...|    76|
|               Brain|    69|
|        Tuberculosis|    68|
|               Cysts|    68|
|          Antibodies|    67|
|           Analgesia|    67|
|            Antigens|    67|
|          Anesthesia|    67|
|Neoplasms, Experi...|    66|
+--------------------+------+
only showing top 20 rows

